In [1]:
#imports
from TCBC_tools import Structure as st, MachineLearning as ml, FeatureExtraction as fe
import numpy as np
import pandas as pd
import multiprocessing as mp
import os
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
pd.options.mode.chained_assignment = None  # default='warn'

/home/tenojo/miniconda3/envs/TCBC-order/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
words_c = []
freqs_c = []
with open("Data_v1-1/whole_csv/Whole.csv") as reader:
    for l in reader.read().split('\n')[1:]:
        t = l.split(";")
        try:
            freqs_c.append(int(t[4]))
            words_c.append(t[0])
        except:
            continue

word_f_corpus = pd.DataFrame({'Word':words_c, 'Freq':freqs_c}).drop_duplicates('Word').sort_values('Freq', ascending=False).reset_index(drop=True)
#Manually fix ", because reading line-by-line has its issues apparently...
word_f_corpus.at[2, 'Word'] = '"'

In [3]:
word_f_corpus['Word'] = word_f_corpus['Word'].apply(lambda x: x.lower())

In [4]:
word_f_corpus = word_f_corpus.drop_duplicates('Word').reset_index(drop=True)

In [5]:
words_p = []
freq_p = []
with open("finnish_vocab.txt") as reader:
    for l in reader:

        t = l.strip().split(" ")
        freq_p.append(int(t[0]))
        words_p.append(t[1])

In [6]:
"""
st_0 = set(words_p[:100])
st_1 = set(words_p[:1000])
st_2 = set(words_p[:5000])
st_3 = set(words_p[:10000])
st_4 = set(words_p[:100000])
st_5 = set(words_p[:1000000])
st_6 = set(words_p[:10000000])
st_7 = set(words_p)

l0, l1, l2, l3, l4, l5, l6, l7 = [],[],[],[],[],[],[],[]

for x in word_f_corpus['Word'].to_numpy().tolist():
    l0.append(x in st_0)
    l1.append(x in st_1)
    l2.append(x in st_2)
    l3.append(x in st_3)
    l4.append(x in st_4)
    l5.append(x in st_5)
    l6.append(x in st_6)
    l7.append(x in st_7)


word_f_corpus['Parsebank'] = l7
word_f_corpus['Parsebank_top_100'] = l0
word_f_corpus['Parsebank_top_1000'] = l1
word_f_corpus['Parsebank_top_5000'] = l2
word_f_corpus['Parsebank_top_10000'] = l3
word_f_corpus['Parsebank_top_100000'] = l4
word_f_corpus['Parsebank_top_1000000'] = l5
word_f_corpus['Parsebank_top_10000000'] = l6

#Garbage collection
del st_0, st_1, st_2, st_3, st_4, st_5, st_6, st_7
"""

"\nst_0 = set(words_p[:100])\nst_1 = set(words_p[:1000])\nst_2 = set(words_p[:5000])\nst_3 = set(words_p[:10000])\nst_4 = set(words_p[:100000])\nst_5 = set(words_p[:1000000])\nst_6 = set(words_p[:10000000])\nst_7 = set(words_p)\n\nl0, l1, l2, l3, l4, l5, l6, l7 = [],[],[],[],[],[],[],[]\n\nfor x in word_f_corpus['Word'].to_numpy().tolist():\n    l0.append(x in st_0)\n    l1.append(x in st_1)\n    l2.append(x in st_2)\n    l3.append(x in st_3)\n    l4.append(x in st_4)\n    l5.append(x in st_5)\n    l6.append(x in st_6)\n    l7.append(x in st_7)\n\n\nword_f_corpus['Parsebank'] = l7\nword_f_corpus['Parsebank_top_100'] = l0\nword_f_corpus['Parsebank_top_1000'] = l1\nword_f_corpus['Parsebank_top_5000'] = l2\nword_f_corpus['Parsebank_top_10000'] = l3\nword_f_corpus['Parsebank_top_100000'] = l4\nword_f_corpus['Parsebank_top_1000000'] = l5\nword_f_corpus['Parsebank_top_10000000'] = l6\n\n#Garbage collection\ndel st_0, st_1, st_2, st_3, st_4, st_5, st_6, st_7\n"

In [7]:
print(np.sum(np.array(freq_p)))

3662727698


In [8]:
qt = dict(zip(words_p, list(range(len(words_p)))))
ranks = [qt.get(x, 50000000) for x in word_f_corpus['Word'].to_numpy()]
word_f_corpus['Parsebank_rank'] = ranks
del ranks
del qt
qt = dict(zip(words_p, freq_p))
freqs = [qt.get(x, 0) for x in word_f_corpus['Word'].to_numpy()]
word_f_corpus['Parsebank_freq'] = freqs
del qt
del freqs
summ = np.sum(np.array(freq_p))
ttt = pd.Series(freq_p, index=words_p)
ggg = fe.getZipfValues(summ, ttt)
qt = dict(zip(list(ggg.index.to_numpy()), ggg.to_numpy()))
ranks = [qt.get(x, -1) for x in word_f_corpus['Word'].to_numpy()]
word_f_corpus['Parsebank_zipf'] = ranks
del summ, ttt, ggg, qt, ranks
del freq_p, words_p

In [9]:
word_f_corpus['Zipf'] = fe.getZipfValues(word_f_corpus['Freq'].sum(), pd.Series(word_f_corpus['Freq'].to_numpy(), word_f_corpus['Word'].to_numpy())).to_numpy()

In [10]:
word_f_corpus['Rank'] = word_f_corpus.index.to_numpy()

In [11]:
full_dataframe = word_f_corpus.iloc[:, [0, 6, 2, 1, 3, 5, 4]]

In [12]:
# 50000000 marks that the word is not in the Parsebank
full_dataframe.sort_values('Zipf')

,Word,Rank,Parsebank_rank,Freq,Parsebank_freq,Zipf,Parsebank_zipf
392887,kuivausrumpu,392887,121426,1,1661,2.015418,2.651564
392906,kuivillakin,392906,1630465,1,46,2.015418,1.103031
392907,kuiviksi oksiksi,392907,50000000,1,0,2.015418,-1.000000
392908,kuivikkeina,392908,1458337,1,54,2.015418,1.171295
392909,kuiviketta,392909,189512,1,931,2.015418,2.400349
...,...,...,...,...,...,...,...
4,on,4,3,242767,78508417,7.099579,7.325849
3,ja,3,2,540524,109102691,7.447204,7.468768
2,"""",2,6,587734,18094912,7.483569,6.688489
1,",",1,1,1223786,188315050,7.802094,7.705818


In [13]:
from scipy.stats import pearsonr
print(pearsonr(full_dataframe['Freq'], full_dataframe['Parsebank_freq']))

PearsonRResult(statistic=np.float64(0.9532416627382755), pvalue=np.float64(0.0))


In [18]:
len(full_dataframe[(full_dataframe['Parsebank_freq'] == 0) & (full_dataframe['Freq']>1)]) / len(full_dataframe[full_dataframe['Freq'] > 1])

0.07694515620748732

In [15]:
print(word_f_corpus['Freq'].sum())
print(word_f_corpus['Parsebank_freq'].sum())

18530894
2910796055


In [16]:
2910796055/3662727698

0.7947071950200978